데이터는
https://www.kaggle.com/competitions/home-credit-default-risk/data
에서 다운

In [174]:
import pandas as pd
import numpy as np

application_train=pd.read_csv(r"C:\Users\USER\Desktop\프로그래밍 공부\파이썬으로 시작하는 캐글\Home Credit Default Risk_CSV\application_train.csv")

In [175]:
application_train.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


각 행 : 대출 신청

Target : 예측 대상

In [176]:
bureau=pd.read_csv(r"C:\Users\USER\Desktop\프로그래밍 공부\파이썬으로 시작하는 캐글\Home Credit Default Risk_CSV\bureau.csv")
bureau.head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN


다른 금융 기관으로부터 제공받은 과거 대출 신청 내역

application_train과 'SK_ID_CURR'이 연결되어있음

1:N의 관계 -> 하나의 대출 신청 레코드와 다수의 과거 대출 신청 내역이 연결되어있음

=> 특징 엔지니어링 필요

과거 신청 횟수가 중요한 특징일 것이라고 가정

In [177]:
previous_loan_counts=\
    bureau.groupby('SK_ID_CURR', 
                   as_index=False)['SK_ID_BUREAU'].count().rename(columns={'SK_ID_BUREAU':'previous_loan_counts'})

# bureau 데이터프레임을 'SK_ID_CURR'을 기준으로 그룹화
# 그룹화된 데이터프레임에서 'SK_ID_BUREAU'열 선택 -> 각 그룹의 레코드 개수 계산
# 만들어진 열의 이름을 'SK_ID_CURR'에서 previous_loan_counts로 변경
previous_loan_counts.head()


# 그룹화된 데이터셋에 COUNT함수 사용 -> 각 그룹의 레코드 개수 반환

,SK_ID_CURR,previous_loan_counts
0,100001,7
1,100002,8
2,100003,4
3,100004,2
4,100005,3


In [178]:
print(application_train['SK_ID_CURR'])
print(len(application_train['SK_ID_CURR']))

print(len(previous_loan_counts['SK_ID_CURR']))
print(previous_loan_counts['SK_ID_CURR'])

0         100002
1         100003
2         100004
3         100006
4         100007
           ...  
307506    456251
307507    456252
307508    456253
307509    456254
307510    456255
Name: SK_ID_CURR, Length: 307511, dtype: int64
307511
305811
0         100001
1         100002
2         100003
3         100004
4         100005
           ...  
305806    456249
305807    456250
305808    456253
305809    456254
305810    456255
Name: SK_ID_CURR, Length: 305811, dtype: int64


In [179]:
matching_ids = previous_loan_counts['SK_ID_CURR'].isin(application_train['SK_ID_CURR'])

# 결과 출력
print("일치하는 ID 수:", sum(matching_ids))
print("불일치 ID 수:", len(matching_ids)- sum(matching_ids))

일치하는 ID 수: 263491
불일치 ID 수: 42320


이후에 SK_ID_CURR을 키로 결합하면 됨

In [180]:
print("previous_loan_counts에서 부족한 ID 수:",len(application_train['SK_ID_CURR'])
          -len(previous_loan_counts['previous_loan_counts']))


previous_loan_counts에서 부족한 ID 수: 1700


In [181]:
application_train=\
    pd.merge(application_train, previous_loan_counts, on='SK_ID_CURR', how='left')

#how ='left' -> 왼쪽 데이터(application_train)을 기준으로 join

In [182]:
print("불일치 ID수 + 부족한 ID수:",42320+1700)
print("결측값 개수 :",application_train['previous_loan_counts'].isnull().sum())
print("application_train 레코드 개수:",len(application_train))

#application_train에 previous_loan_counts를 left join한 결과
#결측값이 발생함 (previous_loan_counts의 아이디와 다른 id, application_train의 sk_id_CURR이 더 많음)

불일치 ID수 + 부족한 ID수: 44020
결측값 개수 : 44020
application_train 레코드 개수: 307511


=> 결측값을 채워줘야함 (DATA SET의 양을 유지해야하기 때문)

맥락상 ID 불일치, ID가 없기 때문이므로 결측값을 0으로 대체

In [185]:
application_train['previous_loan_counts'].fillna(0,inplace=True)

application_train['previous_loan_counts'].isnull().sum()

0